In [138]:
import json
import math
from numpy.linalg import norm
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
import string
from string import digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import cohen_kappa_score as qwk
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.pipeline import Pipeline
from spell import spell_check
from Lemma import lemmatization
from textblob import TextBlob

In [139]:
jawaban = input("Masukkan Jawaban")
kj = input("Masukkan Kunci jawaban")

In [140]:
data = pd.DataFrame([kj])
data.columns = ['Kunci Jawaban']
data['Jawaban'] = jawaban

In [141]:
data

,Kunci Jawaban,Jawaban
0,Fungsi karbohidrat adalah sebagai pemasok ener...,Karbohidrat mempunyai peran penting untuk pros...


In [142]:
print("Text Jawaban: ",jawaban)
print("Text kunci jawaban: ", kj)

Text Jawaban:  Karbohidrat mempunyai peran penting untuk proses metabolisme atau proses yang berfungsi sebagai penyeimbang asam dan basa dalam tubuh. Karbohidrat dapat mencegah terjadinya ketidaksempurnaan proses oksidasi lemak. Fungsi karbohidrat yang utama adalah sebagai pemasok energi. Makanan berkarbohidrat dan serat tinggi dapat membantu memperlancar proses pada pencernaan. 
Text kunci jawaban:  Fungsi karbohidrat adalah sebagai pemasok energi, dapat memperlancar proses pada pencernaan, memberikan efek kenyang dengan kandungan selulosa-nya dan penyeimbang asam dan basa dalam tubuh


In [143]:
#--------------------------------case folding---------------------------------
def case_folding(text):
  pattern = r'[' + string.punctuation + ']'
  punct = re.sub(pattern," ",str(text))
  case_fold = punct.lower()
  return case_fold

def spellcheck(text):
  text = spell_check(text)
  return text

def tokenization(text):
    tokens = re.split(' ',text)
    return tokens

def remove_digits(text):
  text = [item for item in text if item.isalpha()]
  return text

sw= nltk.corpus.stopwords.words('indonesian')
def remove_SW(text):
   text = [item for item in text if not item in sw]
   return text

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
  text = [stemmer.stem(item) for item in text]
  return text

def lemma(text):
  text = lemmatization(text)
  return text


## Pre-Processing

### Case folding

In [144]:
case_fold_q = case_folding(kj)
case_fold_ans = case_folding(jawaban)

print(case_fold_q)
print(case_fold_ans)

fungsi karbohidrat adalah sebagai pemasok energi  dapat memperlancar proses pada pencernaan  memberikan efek kenyang dengan kandungan selulosa nya dan penyeimbang asam dan basa dalam tubuh
karbohidrat mempunyai peran penting untuk proses metabolisme atau proses yang berfungsi sebagai penyeimbang asam dan basa dalam tubuh  karbohidrat dapat mencegah terjadinya ketidaksempurnaan proses oksidasi lemak  fungsi karbohidrat yang utama adalah sebagai pemasok energi  makanan berkarbohidrat dan serat tinggi dapat membantu memperlancar proses pada pencernaan  


### Spell check

In [145]:
spell_q = spell_check(case_fold_q)
spell_ans = spell_check(case_fold_ans)

# print(spell_q)
print("Sebelum: ",jawaban, '\n')
print("Sesudah:", spell_ans)

Sebelum:  Karbohidrat mempunyai peran penting untuk proses metabolisme atau proses yang berfungsi sebagai penyeimbang asam dan basa dalam tubuh. Karbohidrat dapat mencegah terjadinya ketidaksempurnaan proses oksidasi lemak. Fungsi karbohidrat yang utama adalah sebagai pemasok energi. Makanan berkarbohidrat dan serat tinggi dapat membantu memperlancar proses pada pencernaan.  

Sesudah: karbohidrat mempunyai peran penting untuk proses metabolisme atau proses yang berfungsi sebagai penyeimbang asam dan basa dalam tubuh karbohidrat dapat mencegah terjadinya ketidaksempurnaan proses oksidasi lemak fungsi karbohidrat yang utama adalah sebagai pemasok energi makanan berkarbohidrat dan serat tinggi dapat membantu memperlancar proses pada pencernaan


### Tokenization

In [146]:
token_q = tokenization(spell_q)
token_ans = tokenization(spell_ans)
token_q = remove_digits(token_q)
token_ans = remove_digits(token_ans)

print(token_q)
print(token_ans)

['fungsi', 'karbohidrat', 'adalah', 'sebagai', 'pemasok', 'energi', 'dapat', 'memperlancar', 'proses', 'pada', 'pencernaan', 'memberikan', 'efek', 'kenyang', 'dengan', 'kandungan', 'selulosa', 'nya', 'dan', 'penyeimbang', 'asam', 'dan', 'basa', 'dalam', 'tubuh']
['karbohidrat', 'mempunyai', 'peran', 'penting', 'untuk', 'proses', 'metabolisme', 'atau', 'proses', 'yang', 'berfungsi', 'sebagai', 'penyeimbang', 'asam', 'dan', 'basa', 'dalam', 'tubuh', 'karbohidrat', 'dapat', 'mencegah', 'terjadinya', 'ketidaksempurnaan', 'proses', 'oksidasi', 'lemak', 'fungsi', 'karbohidrat', 'yang', 'utama', 'adalah', 'sebagai', 'pemasok', 'energi', 'makanan', 'berkarbohidrat', 'dan', 'serat', 'tinggi', 'dapat', 'membantu', 'memperlancar', 'proses', 'pada', 'pencernaan']


In [147]:
filter_q = remove_SW(token_q)
filter_ans = remove_SW(token_ans)

print(filter_q)
print(filter_ans)

['fungsi', 'karbohidrat', 'pemasok', 'energi', 'memperlancar', 'proses', 'pencernaan', 'efek', 'kenyang', 'kandungan', 'selulosa', 'nya', 'penyeimbang', 'asam', 'basa', 'tubuh']
['karbohidrat', 'peran', 'proses', 'metabolisme', 'proses', 'berfungsi', 'penyeimbang', 'asam', 'basa', 'tubuh', 'karbohidrat', 'mencegah', 'ketidaksempurnaan', 'proses', 'oksidasi', 'lemak', 'fungsi', 'karbohidrat', 'utama', 'pemasok', 'energi', 'makanan', 'berkarbohidrat', 'serat', 'membantu', 'memperlancar', 'proses', 'pencernaan']


### Stemming

In [148]:
stem_q = stemming(filter_q)
stem_ans = stemming(filter_ans)

print(stem_q)
print(stem_ans)

['fungsi', 'karbohidrat', 'pasok', 'energi', 'lancar', 'proses', 'cerna', 'efek', 'kenyang', 'kandung', 'selulosa', 'nya', 'imbang', 'asam', 'basa', 'tubuh']
['karbohidrat', 'peran', 'proses', 'metabolisme', 'proses', 'fungsi', 'imbang', 'asam', 'basa', 'tubuh', 'karbohidrat', 'cegah', 'ketidaksempurnaan', 'proses', 'oksidasi', 'lemak', 'fungsi', 'karbohidrat', 'utama', 'pasok', 'energi', 'makan', 'karbohidrat', 'serat', 'bantu', 'lancar', 'proses', 'cerna']


### Lemmatization

In [149]:
lemma_q = lemma(filter_q)
lemma_ans = lemma(filter_ans)

print(lemma_q)
print(lemma_ans)

['fungsi', 'karbohidrat', 'pemasok', 'energi', 'memperlancar', 'proses', 'pencernaan', 'efek', 'kenyang', 'kandungan', 'selulosa', 'dia', 'penyeimbang', 'asam', 'basa', 'tubuh']
['karbohidrat', 'peran', 'proses', 'metabolisme', 'proses', 'berfungsi', 'penyeimbang', 'asam', 'basa', 'tubuh', 'karbohidrat', 'mencegah', 'tidaksempurna', 'tidaksempurna', 'tidaksempurna', 'tidaksempurna', 'proses', 'oksidasi', 'lemak', 'fungsi', 'karbohidrat', 'utama', 'pemasok', 'energi', 'makanan', 'berkarbohidrat', 'serat', 'membantu', 'memperlancar', 'proses', 'pencernaan']


In [150]:
print(lemma_q)

['fungsi', 'karbohidrat', 'pemasok', 'energi', 'memperlancar', 'proses', 'pencernaan', 'efek', 'kenyang', 'kandungan', 'selulosa', 'dia', 'penyeimbang', 'asam', 'basa', 'tubuh']


In [151]:
print(lemma_ans)

setA = set(lemma_ans)
setQ = set(lemma_q)
print(setA)

words = setA.union(setQ)
print(words,'\n', len(words))

numOfWordsQ = dict.fromkeys(words, 0)
for word in setA:
    numOfWordsQ[word] += 1
    print(numOfWordsQ)


['karbohidrat', 'peran', 'proses', 'metabolisme', 'proses', 'berfungsi', 'penyeimbang', 'asam', 'basa', 'tubuh', 'karbohidrat', 'mencegah', 'tidaksempurna', 'tidaksempurna', 'tidaksempurna', 'tidaksempurna', 'proses', 'oksidasi', 'lemak', 'fungsi', 'karbohidrat', 'utama', 'pemasok', 'energi', 'makanan', 'berkarbohidrat', 'serat', 'membantu', 'memperlancar', 'proses', 'pencernaan']
{'peran', 'serat', 'oksidasi', 'mencegah', 'utama', 'pencernaan', 'fungsi', 'pemasok', 'penyeimbang', 'membantu', 'proses', 'energi', 'makanan', 'basa', 'karbohidrat', 'berfungsi', 'metabolisme', 'tidaksempurna', 'asam', 'lemak', 'tubuh', 'berkarbohidrat', 'memperlancar'}
{'peran', 'serat', 'oksidasi', 'mencegah', 'utama', 'pencernaan', 'fungsi', 'pemasok', 'penyeimbang', 'membantu', 'proses', 'energi', 'makanan', 'basa', 'selulosa', 'karbohidrat', 'berfungsi', 'kenyang', 'dia', 'metabolisme', 'tidaksempurna', 'asam', 'lemak', 'tubuh', 'berkarbohidrat', 'memperlancar', 'kandungan', 'efek'} 
 28
{'peran': 1, '

## TF-IDF + Cosine + Score

In [152]:
def term(q, ans):
    for i in q:
        if i == '':
            q.remove('')
    for i in ans:
        if i == '':
            ans.remove('')
            
    BoWQ = set(q)
    BoWA = set(ans)


    uniqueWords = BoWQ.union(BoWA)
    # print(uniqueWords)

    numOfWordsQ = dict.fromkeys(uniqueWords, 0)
    for word in q:
        numOfWordsQ[word] += 1

    numOfWordsA = dict.fromkeys(uniqueWords, 0)
    for word in ans:
        numOfWordsA[word] += 1
    
    # print('Unique words', numOfWordsA)
        
    term = pd.DataFrame([numOfWordsQ, numOfWordsA])
    term = term.transpose()
    term.columns = ['TF_Q','TF_Ans']

    # display(term)

    dfQ = dict.fromkeys(uniqueWords, 0)
    for word in BoWQ:
        dfQ[word] += 1

    dfA= dict.fromkeys(uniqueWords, 0)
    for word in BoWA:
        dfA[word] += 1
    
    term['DF_Q'] = dfQ.values()
    term['DF_A'] = dfA.values()
    
    DF = []
    for i in range(len(uniqueWords)):
        DF.append(term['DF_Q'][i] + term['DF_A'][i])
    term['DF'] = DF
    # display(term)
    
    idfDict = []
    
    for i in range(len(term['DF'])):
        idfDict.append(math.log10(2 / (term['DF'][i] + 1)))
        # print(idfDict)
    term['IDF'] = idfDict

    # display(term)

    tfidfQ = []
    tfidfA = []
    for i in range(len(uniqueWords)):
        tfidfQ.append(term['TF_Q'][i]*term['IDF'][i])
        tfidfA.append(term['TF_Ans'][i]*term['IDF'][i])

    term['TF-IDF_Q'] = np.array(tfidfQ)
    term['TF-IDF_A'] = np.array(tfidfA)

    cosine = np.dot(tfidfQ,tfidfA)/(np.linalg.norm(tfidfQ)*np.linalg.norm(tfidfA))
    
    if math.isnan(cosine):
        cosine = 0

    print(cosine)
    print(len(term))
    # print('Skor: ', round((cosine*100),2))
    # print(len(term))

    return term

In [153]:
term(tokenization(jawaban), tokenization(kj))

0.8655784229010972
46


,TF_Q,TF_Ans,DF_Q,DF_A,DF,IDF,TF-IDF_Q,TF-IDF_A
dalam,1,1,1,1,2,-0.176091,-0.176091,-0.176091
peran,1,0,1,0,1,0.000000,0.000000,0.000000
adalah,1,1,1,1,2,-0.176091,-0.176091,-0.176091
yang,2,0,1,0,1,0.000000,0.000000,0.000000
serat,1,0,1,0,1,0.000000,0.000000,0.000000
tinggi,1,0,1,0,1,0.000000,0.000000,0.000000
Karbohidrat,2,0,1,0,1,0.000000,0.000000,0.000000
oksidasi,1,0,1,0,1,0.000000,0.000000,0.000000
mencegah,1,0,1,0,1,0.000000,0.000000,0.000000
utama,1,0,1,0,1,0.000000,0.000000,0.000000


In [154]:
term(stem_q, stem_ans)

0.818181818181818
26


,TF_Q,TF_Ans,DF_Q,DF_A,DF,IDF,TF-IDF_Q,TF-IDF_A
basa,1,1,1,1,2,-0.176091,-0.176091,-0.176091
peran,0,1,0,1,1,0.000000,0.000000,0.000000
selulosa,1,0,1,0,1,0.000000,0.000000,0.000000
cegah,0,1,0,1,1,0.000000,0.000000,0.000000
karbohidrat,1,4,1,1,2,-0.176091,-0.176091,-0.704365
serat,0,1,0,1,1,0.000000,0.000000,0.000000
makan,0,1,0,1,1,0.000000,0.000000,0.000000
kenyang,1,0,1,0,1,0.000000,0.000000,0.000000
oksidasi,0,1,0,1,1,0.000000,0.000000,0.000000
kandung,1,0,1,0,1,0.000000,0.000000,0.000000


In [155]:
term(lemma_q, lemma_ans)

0.8273403039920308
28


,TF_Q,TF_Ans,DF_Q,DF_A,DF,IDF,TF-IDF_Q,TF-IDF_A
peran,0,1,0,1,1,0.000000,0.000000,0.000000
serat,0,1,0,1,1,0.000000,0.000000,0.000000
oksidasi,0,1,0,1,1,0.000000,0.000000,0.000000
mencegah,0,1,0,1,1,0.000000,0.000000,0.000000
utama,0,1,0,1,1,0.000000,0.000000,0.000000
pencernaan,1,1,1,1,2,-0.176091,-0.176091,-0.176091
fungsi,1,1,1,1,2,-0.176091,-0.176091,-0.176091
pemasok,1,1,1,1,2,-0.176091,-0.176091,-0.176091
penyeimbang,1,1,1,1,2,-0.176091,-0.176091,-0.176091
membantu,0,1,0,1,1,0.000000,0.000000,0.000000


## Norm?

In [156]:
import math
from numpy.linalg import norm

def term_ori(query, answer):
    #--------------------term-----------------
    print(len(query))
    print(len(answer))
    BoWQ = set(query)
    BoWA = set(answer)
    uniqueWords = BoWQ.union(BoWA)

    #-----term frequency of word in doc-------
    numOfWordsQ = dict.fromkeys(uniqueWords, 0)
    for word in query:
        numOfWordsQ[word] += 1

    numOfWordsA = dict.fromkeys(uniqueWords, 0)
    for word in answer:
        numOfWordsA[word] += 1
    
    # print(numOfWordsA)
    term = pd.DataFrame([numOfWordsQ, numOfWordsA])
    term = term.transpose()
    term.columns = ['term_q','term_ans']

    #--------------term frequency (TF)---------------
    tfQ = {}
    tfA = {}

    bagOfWordsCount = len(query)
    #tf Kunci Jawaban
    for word, count in numOfWordsQ.items():
        tfQ[word] = count / float(bagOfWordsCount)

    #tf Jawaban
    for word, count in numOfWordsA.items():
        if len(answer) == 0:
            tfA[word] = 0
        else:
            tfA[word] = count / float(len(answer))
   
    term['TF_Q'] = tfQ.values()
    term['TF_Ans'] = tfA.values()

    dfQ = dict.fromkeys(uniqueWords, 0)
    for word in BoWQ:
        dfQ[word] += 1

    dfA= dict.fromkeys(uniqueWords, 0)
    for word in BoWA:
        dfA[word] += 1
    
    DF = []
    for i in range(len(uniqueWords)):
        DF.append(list(dfQ.values())[i]+list(dfA.values())[i])
    term['DF'] = DF

    #-------------------IDF---------------------------
    N = len([numOfWordsQ, numOfWordsA])
    
    # idfDict = dict.fromkeys([numOfWordsQ, numOfWordsA][0].keys(), 0)
    # for document in [numOfWordsQ, numOfWordsA]:
    #     for word, val in document.items():
    #         if val > 0:
    #             idfDict[word] += 1
    
    # for word, val in idfDict.items():
    #     idfDict[word] = math.log(N+1 / float(val+1))

    idf = []
    for i in range(len(DF)):
        idf.append(math.log10(N / (DF[i]+1)))
        # print(val)

    term['IDF'] = idf

    #-------------------TF-IDF---------------------
    tfidfQ = []
    tfidfA = []
    
    for i in range(len(uniqueWords)):
        tfidfQ.append(list(tfQ.values())[i]*idf[i])
        tfidfA.append(list(tfA.values())[i]*idf[i])
    
    # tfidfQ = list(tfidfQ)
    # tfidfA = list(tfidfA)
    term['TF_IDF Q'] = tfidfQ
    term['TF_IDF A'] = tfidfA

    cosine =np.dot(tfidfQ,tfidfA)/(np.linalg.norm(tfidfQ)*np.linalg.norm(tfidfA))
    print(len(tfidfA))
    print(cosine)
    print('Skor: ', round((cosine*100),2))
    global SkorSCT
    SkorSCT = str(round((cosine*100), 2))
    
    
    return term


In [157]:
term_ori(kj.split(), jawaban.split())

24
45
46
0.8655784229010972
Skor:  86.56


,term_q,term_ans,TF_Q,TF_Ans,DF,IDF,TF_IDF Q,TF_IDF A
adalah,1,1,0.041667,0.022222,2,-0.176091,-0.007337,-0.003913
dalam,1,1,0.041667,0.022222,2,-0.176091,-0.007337,-0.003913
peran,0,1,0.000000,0.022222,1,0.000000,0.000000,0.000000
yang,0,2,0.000000,0.044444,1,0.000000,0.000000,0.000000
serat,0,1,0.000000,0.022222,1,0.000000,0.000000,0.000000
tinggi,0,1,0.000000,0.022222,1,0.000000,0.000000,0.000000
Karbohidrat,0,2,0.000000,0.044444,1,0.000000,0.000000,0.000000
oksidasi,0,1,0.000000,0.022222,1,0.000000,0.000000,0.000000
mencegah,0,1,0.000000,0.022222,1,0.000000,0.000000,0.000000
utama,0,1,0.000000,0.022222,1,0.000000,0.000000,0.000000


In [158]:
print(stem_q)
print(stem_ans)

['fungsi', 'karbohidrat', 'pasok', 'energi', 'lancar', 'proses', 'cerna', 'efek', 'kenyang', 'kandung', 'selulosa', 'nya', 'imbang', 'asam', 'basa', 'tubuh']
['karbohidrat', 'peran', 'proses', 'metabolisme', 'proses', 'fungsi', 'imbang', 'asam', 'basa', 'tubuh', 'karbohidrat', 'cegah', 'ketidaksempurnaan', 'proses', 'oksidasi', 'lemak', 'fungsi', 'karbohidrat', 'utama', 'pasok', 'energi', 'makan', 'karbohidrat', 'serat', 'bantu', 'lancar', 'proses', 'cerna']


In [159]:
term_ori(stem_q, stem_ans)

16
28
26
0.8181818181818182
Skor:  81.82


,term_q,term_ans,TF_Q,TF_Ans,DF,IDF,TF_IDF Q,TF_IDF A
basa,1,1,0.0625,0.035714,2,-0.176091,-0.011006,-0.006289
peran,0,1,0.0000,0.035714,1,0.000000,0.000000,0.000000
selulosa,1,0,0.0625,0.000000,1,0.000000,0.000000,0.000000
cegah,0,1,0.0000,0.035714,1,0.000000,0.000000,0.000000
karbohidrat,1,4,0.0625,0.142857,2,-0.176091,-0.011006,-0.025156
serat,0,1,0.0000,0.035714,1,0.000000,0.000000,0.000000
makan,0,1,0.0000,0.035714,1,0.000000,0.000000,0.000000
kenyang,1,0,0.0625,0.000000,1,0.000000,0.000000,0.000000
oksidasi,0,1,0.0000,0.035714,1,0.000000,0.000000,0.000000
kandung,1,0,0.0625,0.000000,1,0.000000,0.000000,0.000000


In [160]:
term_ori(lemma_q, lemma_ans)

16
31
28
0.8273403039920307
Skor:  82.73


,term_q,term_ans,TF_Q,TF_Ans,DF,IDF,TF_IDF Q,TF_IDF A
peran,0,1,0.0000,0.032258,1,0.000000,0.000000,0.000000
serat,0,1,0.0000,0.032258,1,0.000000,0.000000,0.000000
oksidasi,0,1,0.0000,0.032258,1,0.000000,0.000000,0.000000
mencegah,0,1,0.0000,0.032258,1,0.000000,0.000000,0.000000
utama,0,1,0.0000,0.032258,1,0.000000,0.000000,0.000000
pencernaan,1,1,0.0625,0.032258,2,-0.176091,-0.011006,-0.005680
fungsi,1,1,0.0625,0.032258,2,-0.176091,-0.011006,-0.005680
pemasok,1,1,0.0625,0.032258,2,-0.176091,-0.011006,-0.005680
penyeimbang,1,1,0.0625,0.032258,2,-0.176091,-0.011006,-0.005680
membantu,0,1,0.0000,0.032258,1,0.000000,0.000000,0.000000


In [162]:
newfile = open("HasilPenilaian.json", "w")

newfile.write('{ "keyAnswer": "' + kj + '", \n"studentAnswer": "' +
              jawaban + '", \n"scoringSCT": "' + SkorSCT + '"}')


619